# DBSCAN Post Processing

**In this notebook:**

* Test DBSCAN for post processing

## Dependencies
Install, load, and initialize all required dependencies for this experiment.

### Install Dependencies

In [ ]:
#It should be possible to run the notebook independent of anything else. 
# If dependency cannot be installed via pip, either:
# - download & install it via %%bash
# - atleast mention those dependecies in this section

import sys
!{sys.executable} -m pip install -q -e ../../utils/

# System libraries

In [ ]:
from __future__ import absolute_import, division, print_function
import logging, os, sys

# Enable logging
logging.basicConfig(format='[%(levelname)s] %(message)s', level=logging.INFO, stream=sys.stdout)

# Re-import packages if they change
%load_ext autoreload
%autoreload 2

# Recursion Depth
import sys
sys.setrecursionlimit(10000)

# Intialize tqdm to always use the notebook progress bar
import tqdm
tqdm.tqdm = tqdm.tqdm_notebook

# Third-party libraries
import comet_ml
import numpy as np
import pandas as pd
import nilearn.plotting as nip
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import collections
%matplotlib inline
plt.rcParams["figure.figsize"] = (12,6)
%config InlineBackend.figure_format='retina'  # adapt plots for retina displays
import git


# Project utils

import aneurysm_utils
from aneurysm_utils import evaluation, training


In [ ]:
os.getcwd()

### Initialize Environment

In [ ]:
env = aneurysm_utils.Environment(project="ML_IN_MIP", root_folder="/workspace/")
env.cached_data["comet_key"] = "EGrR4luSis87yhHbs2rEaqAWs" 
env.print_info()

## Load Data
Download, explore, and prepare all required data for the experiment in this section.

In [ ]:
dataset_params = {
    "prediction": "LabeledMask",
    "mri_data_selection": "unprocessed", 
    "balance_data": False,
    "seed": 1,
    "resample_voxel_dim": None
}

preprocessing_params = {
    'min_max_normalize': None,
    'mean_std_normalize': False,
    'smooth_img': False, # can contain a number: smoothing factor
}


### Load Meta Data

In [ ]:
from aneurysm_utils.data_collection import load_aneurysm_dataset

df = load_aneurysm_dataset(
    env,
    mri_data_selection=dataset_params["mri_data_selection"],
    random_state=dataset_params["seed"]
)
df.head()

In [ ]:
from aneurysm_utils import data_collection 


In [ ]:
img_dict = data_collection.get_case_images(env, df, "A130_R", mesh=False, resample_voxel_dim=dataset_params["resample_voxel_dim"])

In [ ]:
img_dict.keys()

In [ ]:
nip.view_img(
    img_dict["Mask nii"], 
    symmetric_cmap=False,
    cmap="Greys_r",
    bg_img=False,
    black_bg=True,
    threshold=1e-03, 
    draw_cross=False,
)

In [ ]:
np.unique(img_dict["Labeled Mask struct_arr"])

In [ ]:
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=1, min_samples=5).fit(np.array(np.where(img_dict["Mask struct_arr"]==1)).T)



In [ ]:
np.unique(db.labels_)

In [ ]:
dbscan_mask = np.zeros((img_dict["Mask struct_arr"].shape))
for i, label in zip(np.array(np.where(img_dict["Mask struct_arr"]==1)).T, db.labels_):
    dbscan_mask[i[0], i[1], i[2]] = label+1


In [ ]:
nip.view_img(
    nib.Nifti1Image(dbscan_mask, np.eye(4)), 
    symmetric_cmap=False,
    bg_img=False,
    black_bg=True,
    threshold=1e-03, 
    draw_cross=False,
)

In [ ]:
nip.view_img(
    img_dict["Labeled Mask nii"], 
    symmetric_cmap=False,
    bg_img=False,
    black_bg=True,
    threshold=1e-03, 
    draw_cross=False,
)

In [ ]:
dbscan_sets = []
mask_sets = []
for array in [dbscan_mask, mask]:
    for i in np.unique(dbscan_mask).tolist():
        aneurysm_index = np.array(np.where(dbscan_mask==(i-1))).T
        x_hashable = map(tuple, aneurysm_index)
        y = set(x_hashable)
        dbscan_sets.append(y)
        print(y)